# Laste ned og analysere åpne data fra Enhetsregisteret

Søke-API-et kan brukes for å gjøre forespørsler i Enhetsregisteret, og få resultatene direkte. Men API-et har en begrensning som gjør at det ikke er mulig å få mer enn 10.000 enheter i resultatet. Hvis du for eksempel søker etter enheter med et sett av næringskoder, og resultatet er mer enn 10.000, får du kun tilgang til detaljene om de 10.000 første enhetene i resultatet.

Det er derfor nyttig å kunne laste ned hele datasettet og jobbe med en kopi. I denne notatboken viser vi hvordan du 1) laster ned hele Enhetsregisteret, 2) Konverterer filen til CSV-format (fordi det går raskere å jobbe med senere), 3) leser filen med Python og Pandas, og til slutt 4) noen eksempler på enkle analyser.

Merk at siden registeret er såpass stort tar det noen minutter både å laste ned og konvertere den nedlastede fila. Hvis du ikke er avhengig av å bruke helt oppdaterte data, kan du bruke den vedlagte kopien. Isåfall kan du hoppe til [Alternativ løsning](#alternativ).

Denne notatboken bruker Python, og all koden skal fungere i alle moderne python-miljø. I tillegg til Python bruker vi følgende verktøy, som må installeres før vi går videre:

- requests: Et verktøy for kommunisere med data.brreg.no
- xlsx2csv: Et verktøy for å konvertere fra *.xlsx-formatet til *.csv-format
- pandas: Et populært verktøy for å håndtere data, inkludert "rydde" og analysere dem




In [1]:
!pip install requests
!pip install xlsx2csv
!pip install pandas

  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached urllib3-1.26.4-py2.py3-none-any.whl (153 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
     |████████████████████████████████| 230 kB 3.7 MB/s 
Using legacy 'setup.py install' for xlsx2csv, since package 'wheel' is not installed.
    Running setup.py install for xlsx2csv ... done
  Using cached pandas-1.2.4-cp39-cp39-manylinux1_x86_64.whl (9.7 MB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
  Using cached numpy-1.20.2-cp39-cp39-manylinux2010_x86_64.whl (15.4 MB)


Etter at verktøyene er installert, må de importeres for å være tilgjengelige for Python.

In [7]:
import requests
from xlsx2csv import Xlsx2csv
import pandas as pd
import numpy as np

## 1) Laste ned alle hovedenheter fra Enhetsregisteret
Hovedenhetene kan lastes ned som enten JSON- eller XLSX-format. Vi bruker XLSX-formatet fordi det er er "forflatet" til en stor tabell, noe som gjør det enklere å bruke med Pandas-verktøyet.

Filen som lastes ned er ca 200 MB stor. Hvor langt tid det tar vil være avhengig av nettverket. Hvis kjører koden fra mybinder.org tar det erfaringsmessig i underkant av fem minutter. På en PC på hjemmekontor har det tatt opp mot 10 minutter.

Det er mulig å laste ned og lagre en fil med enklere kode enn det som er skrevet nedenfor, men da lastes hele filen ned i minnet før den lagres. Det krever mer minne.

NB! MyBinder.org er en gratistjeneste, og for å redusere behovet for maskinressurser, skrur den av virtuelle maskiner som ikke er i bruk, det vil si at all koden du har kjørt blir nullstilt [hvis du er inaktiv i mer enn ti minutter](https://mybinder.readthedocs.io/en/latest/about/about.html#how-long-will-my-binder-session-last). **Da må du starte maskinen på nytt, og sannsynligvis må all kode må kjøres på nytt og fila må lastes ned på nytt.**


In [3]:
%%time
url = 'https://data.brreg.no/enhetsregisteret/api/enheter/lastned/regneark'
headers = {'Accept': 'application/vnd.brreg.enhetsregisteret.enhet+vnd.openxmlformats-officedocument.spreadsheetml.sheet;charset=UTF-8'}
session = requests.Session() # establish a session that is kept open during the transfer, instead of performing separate requests
r = session.get(url, headers=headers, stream = True)
r.raise_for_status()
with open('er.xlsx','wb') as f:
    for chunk in r.iter_content(1024*1024*2): # laster ned og skriver ca 2 MB av gangen
        f.write(chunk)

CPU times: user 2.11 s, sys: 1.68 s, total: 3.79 s
Wall time: 8min 7s


# 2) Konvertere filen til CSV-formatet
Python Pandas har en funksjon for å lese xlsx-filer, men den er tregere og krever mer minne enn tilsvarende metode for csv-filer. Du sparer derfor tid på å konvertere filen til csv-formatet først. Når du kjører koden på mybinder.org, risikerer du dessuten at lesing av xlsx-filen krever mer minne enn du er tildelt, og da stopper prosessen.

For å konvertere den nedlastede fila, bruker vi xlsx2csv. På mybinder.org tar dette erfaringsmessig i underkant av 5 minutter. På en PC på hjemmekontor tar det nærmere ti.


In [4]:
%time Xlsx2csv("er.xlsx", outputencoding="utf-8").convert("er.csv")

CPU times: user 8min 6s, sys: 12.2 s, total: 8min 18s
Wall time: 8min 19s


## 3) Lese filen med Python og Pandas

Pandas har en egen funksjon for å lese csv-filer, '''read_csv'''. I kommandoen nedenfor har vi i tillegg spesifisert informasjon om typene data i filen, for å gjøre det lettere å bruke senere.

In [8]:
df = pd.read_csv('er.csv', dtype={
        'Organisasjonsnummer': str,
        'Navn': str,
        'Organisasjonsform.kode': 'category',
        'Organisasjonsform.beskrivelse': 'category',
        'Næringskode 1': str,
        'Næringskode 1.beskrivelse': str,
        'Næringskode 2': str,
        'Næringskode 2.beskrivelse': str,
        'Næringskode 3': str,
        'Næringskode 3.beskrivelse': str,
        'Hjelpeenhetskode': 'category',
        'Hjelpeenhetskode.beskrivelse': 'category',
        'Antall ansatte': np.int16,
        'Hjemmeside': str,
        'Postadresse.adresse': str,
        'Postadresse.poststed': str,
        'Postadresse.postnummer': str,
        'Postadresse.kommune': str,
        'Postadresse.kommunenummer': str,
        'Postadresse.land': 'category',
        'Postadresse.landkode': 'category',
        'Forretningsadresse.adresse': str,
        'Forretningsadresse.poststed': str,
        'Forretningsadresse.postnummer': str,
        'Forretningsadresse.kommune': str,
        'Forretningsadresse.kommunenummer': str,
        'Forretningsadresse.land': 'category',
        'Forretningsadresse.landkode': 'category',
        'Institusjonell sektorkode': 'category',
        'Institusjonell sektorkode.beskrivelse': 'category',
        'Siste innsendte årsregnskap': str, # klarte ikke konvertere til np.int16
        'Registreringsdato i Enhetsregisteret': str, # klarer ikke konvertere 'datetime64',
        'Stiftelsesdato': str, # klarte ikke å konvertere til datetime64 - 1550-12-31 00:00:00
        'FrivilligRegistrertIMvaregisteret': 'category',
        'Registrert i MVA-registeret': 'category',
        'Registrert i Frivillighetsregisteret': 'category',
        'Registrert i Foretaksregisteret': 'category',
        'Registrert i Stiftelsesregisteret': 'category',
        'Konkurs': 'category',
        'Under avvikling': 'category',
        'Under tvangsavvikling eller tvangsoppløsning': 'category',
        'Overordnet enhet i offentlig sektor': str,
        'Målform': 'category' })

<a id='alternativ'></a>
# Alternativ: Bruk vedlagte kopi

Hvis du ikke er avhengig av helt oppdaterte data, ligger det en kopi av er.csv som del av dette arbeidsområdet. Ved å bruke den sparer du tid på å laste ned og konvertere fila. For informasjon tidspunkt for fila, kjør kommandoen nedenfor:

In [5]:
!ls -l er.csv.gz

-rw-r--r-- 1 wslstsk wslstsk 64931668 Apr 29 13:31 er.csv.gz


NB! Ikke kjør denne cellen med kode hvis du allerede har lastet ned og lest inn oppdaterte data :-) 


In [9]:
df = pd.read_csv('er.csv.gz', compression='gzip', dtype={
        'Organisasjonsnummer': str,
        'Navn': str,
        'Organisasjonsform.kode': 'category',
        'Organisasjonsform.beskrivelse': 'category',
        'Næringskode 1': str,
        'Næringskode 1.beskrivelse': str,
        'Næringskode 2': str,
        'Næringskode 2.beskrivelse': str,
        'Næringskode 3': str,
        'Næringskode 3.beskrivelse': str,
        'Hjelpeenhetskode': 'category',
        'Hjelpeenhetskode.beskrivelse': 'category',
        'Antall ansatte': np.int16,
        'Hjemmeside': str,
        'Postadresse.adresse': str,
        'Postadresse.poststed': str,
        'Postadresse.postnummer': str,
        'Postadresse.kommune': str,
        'Postadresse.kommunenummer': str,
        'Postadresse.land': 'category',
        'Postadresse.landkode': 'category',
        'Forretningsadresse.adresse': str,
        'Forretningsadresse.poststed': str,
        'Forretningsadresse.postnummer': str,
        'Forretningsadresse.kommune': str,
        'Forretningsadresse.kommunenummer': str,
        'Forretningsadresse.land': 'category',
        'Forretningsadresse.landkode': 'category',
        'Institusjonell sektorkode': 'category',
        'Institusjonell sektorkode.beskrivelse': 'category',
        'Siste innsendte årsregnskap': str, # klarte ikke konvertere til np.int16
        'Registreringsdato i Enhetsregisteret': str, # klarer ikke konvertere 'datetime64',
        'Stiftelsesdato': str, # klarte ikke å konvertere til datetime64 - 1550-12-31 00:00:00
        'FrivilligRegistrertIMvaregisteret': 'category',
        'Registrert i MVA-registeret': 'category',
        'Registrert i Frivillighetsregisteret': 'category',
        'Registrert i Foretaksregisteret': 'category',
        'Registrert i Stiftelsesregisteret': 'category',
        'Konkurs': 'category',
        'Under avvikling': 'category',
        'Under tvangsavvikling eller tvangsoppløsning': 'category',
        'Overordnet enhet i offentlig sektor': str,
        'Målform': 'category' })